In [13]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
split_unit = "6M"
modulename = "BacktestingStrategies.Strategy_BBDandRSI"
classname = "btst.BBandRSI"
end = datetime.date.today()

data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [14]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [15]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [16]:
import statistics

df = pd.DataFrame(returns)

#最低と最大の期間のBacktestingを表示＆実行
lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

#リターンが最小のものの期間を取得
min_return = df[df["# Trades"] != 0]["Return [%]"].min()
min_period_start = pd.to_datetime(df[df["Return [%]"] == min_return]["Start"]).values[0]
min_period_end = pd.to_datetime(df[df["Return [%]"] == min_return]["End"]).values[0]
df_min = data[min_period_start : min_period_end]

#リターンが2番目、3番目に小さいものの期間を取得
min_return_3 = df[df["# Trades"] != 0]["Return [%]"].nsmallest(3)
min2_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["Start"]).values[0]
min2_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["End"]).values[0]
min3_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["Start"]).values[0]
min3_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["End"]).values[0]
df_min2 = data[min2_period_start : min2_period_end]
df_min3 = data[min3_period_start : min3_period_end]

#リターンが最大のものの期間を取得
max_return = df[df["# Trades"] != 0]["Return [%]"].max()
max_period_start = pd.to_datetime(df[df["Return [%]"] == max_return]["Start"]).values[0]
max_period_end = pd.to_datetime(df[df["Return [%]"] == max_return]["End"]).values[0]
df_max = data[max_period_start : max_period_end]

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 

print(lines)



１期間単位: 6M
テスト期間: 11
平均Return [%]: 1.699
最大Return [%]: 18.108653099999998
最大Return期間: 2019-04-01T00:00:00.000000000 - 2019-09-30T00:00:00.000000000
最低Return [%]: -27.438317499999993
最低Return期間: 2019-10-01T00:00:00.000000000 - 2020-03-31T00:00:00.000000000


In [17]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()

In [18]:
#最低と最高の期間のバックテストを表示

print("----------WORST----------")
bt = Backtest(
    df_min, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min.html") # 実行結果（グラフ）

print("----------MIN2----------")
bt = Backtest(
    df_min2, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min2.html") # 実行結果（グラフ）

print("----------MIN3----------")
bt = Backtest(
    df_min3, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min3.html") # 実行結果（グラフ）

print("----------BEST----------")
bt = Backtest(
    df_max, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_max.html") # 実行結果（グラフ）

----------WORST----------
Start                     2019-10-01 00:00:00
End                       2020-03-31 00:00:00
Duration                    182 days 00:00:00
Exposure Time [%]                   19.166667
Equity Final [$]                   725616.825
Equity Peak [$]                     1000000.0
Return [%]                         -27.438317
Buy & Hold Return [%]               -27.88162
Return (Ann.) [%]                  -49.009952
Volatility (Ann.) [%]               26.571279
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -46.088317
Avg. Drawdown [%]                  -46.088317
Max. Drawdown Duration       34 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                     -27.676804
Worst Trade [%]                    -27.676804
Avg. Tra

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,50,97,119,19827.6635,14340.0,-274383.175,-0.276768,2020-02-27,2020-03-31,33 days


----------MIN2----------
Start                     2017-10-02 00:00:00
End                       2018-03-30 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                   29.230769
Equity Final [$]                   928136.625
Equity Peak [$]                   1008636.625
Return [%]                          -7.186338
Buy & Hold Return [%]               10.023866
Return (Ann.) [%]                  -13.459991
Volatility (Ann.) [%]               21.866732
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -15.714282
Avg. Drawdown [%]                   -12.80031
Max. Drawdown Duration       31 days 00:00:00
Avg. Drawdown Duration       26 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -7.278311
Worst Trade [%]                     -7.278311
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,50,92,129,19747.2675,18310.0,-71863.375,-0.072783,2018-02-07,2018-03-30,51 days


----------MIN3----------
Start                     2018-10-01 00:00:00
End                       2019-03-29 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                   52.419355
Equity Final [$]                   1097752.96
Equity Peak [$]                    1143352.96
Return [%]                           9.775296
Buy & Hold Return [%]              -24.642557
Return (Ann.) [%]                   20.869252
Volatility (Ann.) [%]               38.078346
Sharpe Ratio                         0.548061
Sortino Ratio                        0.956361
Calmar Ratio                         1.810828
Max. Drawdown [%]                  -11.524704
Avg. Drawdown [%]                   -3.419411
Max. Drawdown Duration       25 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       9.933769
Worst Trade [%]                      9.933769
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,60,59,123,16400.784,18030.0,97752.96,0.099338,2018-12-21,2019-03-29,98 days


----------BEST----------
Start                     2019-04-01 00:00:00
End                       2019-09-30 00:00:00
Duration                    182 days 00:00:00
Exposure Time [%]                   30.578512
Equity Final [$]                  1181086.531
Equity Peak [$]                   1204026.531
Return [%]                          18.108653
Buy & Hold Return [%]                2.818428
Return (Ann.) [%]                   41.428563
Volatility (Ann.) [%]               20.955468
Sharpe Ratio                         1.976981
Sortino Ratio                        4.888831
Calmar Ratio                         9.539645
Max. Drawdown [%]                   -4.342778
Avg. Drawdown [%]                   -1.839357
Max. Drawdown Duration       14 days 00:00:00
Avg. Drawdown Duration        7 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      18.153429
Worst Trade [%]                     18.153429
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,62,84,120,16089.2495,19010.0,181086.531,0.181534,2019-08-06,2019-09-30,55 days


Row(id='15358', ...)